In [36]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
import keras.losses
import matplotlib.pyplot as plt
import tensorflow as tf
import math
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import random as rnd
from keras.callbacks import EarlyStopping
from keras.callbacks import Callback

In [16]:
def normalize(data):
    tmp = [((item - min(data)) / ((max(data)-min(data)))) for item in data]
    return np.array(tmp)

In [17]:
data = pd.read_csv('khodro.csv')
closePrice, volume = normalize(data['Close']), normalize(data['Volume'])

In [18]:
compact = [[p, v] for p, v in zip(closePrice, volume)]
clips = []
nextPred = []
period = 5
for i in range(period, len(closePrice)):
    clips.append(compact[i-period:i])


for clipp in clips[1:]:
    if clipp[-1][0] > clipp[0][0]:
        nextPred.append([1,0,0])
    elif clipp[-1][0] == clipp[0][0]:
        nextPred.append([0,0,1])
    else:
        nextPred.append([0,1,0])
clips.pop(-1)


[[0.11211026136650139, 0.08975190957669205],
 [0.10664761744684427, 0.07381259416689671],
 [0.10093285150012606, 0.10551980588683692],
 [0.10320194974367594, 0.13904240536248844],
 [0.1072359021766535, 0.07369322173610628]]

In [19]:
xTrain, xTest, yTrain, yTest = train_test_split(clips, nextPred, test_size=0.1, shuffle=False)

In [20]:
model = keras.Sequential()
model.add(layers.Flatten(input_shape=(period,2)))
model.add(layers.Dense(100))
model.add(layers.Dense(50))
model.add(layers.Dense(10))
model.add(layers.Dense(5))
model.add(layers.Dense(3))
model.add(layers.Activation('softmax'))

model.compile(optimizer='Adam', loss=keras.losses.CategoricalCrossentropy(), metrics=keras.metrics.CategoricalAccuracy())
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 10)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 100)               1100      
_________________________________________________________________
dense_6 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_7 (Dense)              (None, 10)                510       
_________________________________________________________________
dense_8 (Dense)              (None, 5)                 55        
_________________________________________________________________
dense_9 (Dense)              (None, 3)                 18        
_________________________________________________________________
activation_1 (Activation)    (None, 3)                

In [38]:
class TerminateOnBaseline(Callback):
    def __init__(self, monitor='categorical_accuracy', baseline=0.9):
        super(TerminateOnBaseline, self).__init__()
        self.monitor = monitor
        self.baseline = baseline

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        acc = logs.get(self.monitor)
        if acc is not None:
            if acc >= self.baseline:
                print('Epoch %d: Reached baseline, terminating training' % (epoch))
                self.model.stop_training = True

In [55]:
callbacks = [TerminateOnBaseline(monitor='categorical_accuracy', baseline=0.85)]

In [56]:
resss = model.fit(xTrain, yTrain, batch_size=128, epochs=100, callbacks=[callbacks])

Epoch 1/100
21/21 [==============================] - 0s 2ms/step - loss: 0.4531 - categorical_accuracy: 0.8153
Epoch 2/100
21/21 [==============================] - 0s 3ms/step - loss: 0.4395 - categorical_accuracy: 0.8309
Epoch 3/100
21/21 [==============================] - 0s 3ms/step - loss: 0.4410 - categorical_accuracy: 0.8301
Epoch 4/100
21/21 [==============================] - 0s 3ms/step - loss: 0.4471 - categorical_accuracy: 0.8145
Epoch 5/100
21/21 [==============================] - 0s 3ms/step - loss: 0.4379 - categorical_accuracy: 0.8270
Epoch 6/100
21/21 [==============================] - 0s 4ms/step - loss: 0.4492 - categorical_accuracy: 0.8118
Epoch 7/100
21/21 [==============================] - 0s 5ms/step - loss: 0.4471 - categorical_accuracy: 0.8180
Epoch 8/100
21/21 [==============================] - 0s 5ms/step - loss: 0.4586 - categorical_accuracy: 0.8091
Epoch 9/100
21/21 [==============================] - 0s 4ms/step - loss: 0.4473 - categorical_accuracy: 0.8215
E